Toy GROTH16 implementation 

In [37]:
import numpy as np
import galois
from functools import reduce
from py_ecc.bn128 import G1, G2, multiply, add, curve_order, Z1, pairing, neg, final_exponentiate, FQ12

Trusted setup parameters.

In [88]:
# Using BN.128 curve order.
GF = galois.GF(curve_order)

We want to show that we know the solution to out = xˆ4-5yxˆ2

In [ ]:
# La * Ra = Oa

# Left Matrix
L = np.array([
    [0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, curve_order-5, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 1],
])

# Right matrix 
R = np.array([
    [0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0],
])

# Output matrix 
O = np.array([
    [0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 0, 1],
    [0, 1, 0, 0, 0, curve_order-1, 0],
])

# Converting matrices into out Galois Field of BN.128 order
L_galois = GF(L)
R_galois = GF(R)
O_galois = GF(O)

# Witness vector (Secret solution)
x = GF(2)
y = GF(7)
v1 = GF(4)
v2 = GF(16)
v3 = GF(curve_order-245)
out = GF(curve_order-964)

a = GF(np.array([1, out, x, y, v1, v2, v3]))

# Sanity check for our contraint system
assert all(np.equal(np.matmul(L_galois, a) * np.matmul(R_galois, a), np.matmul(O_galois, a))), "R1CS constraint violation"

# Splitting witness vector into public and private 
l = 1  # Index for the last public input, 1 and out are public 
public_inputs = a[:l+1]
private_inputs = a[l+1:]

In [91]:
# Transform R1CS to QAP 
def interpolate_matrix_column(col):
    # interpolating every matrix column with a common vector (1,2,3,...,n)
    xs = GF(np.array(range(1, len(col) + 1)))
    return galois.lagrange_poly(xs, col)


# Polynomials used for QAP 
U_polys = np.apply_along_axis(interpolate_column_galois, 0, L_galois)
V_polys = np.apply_along_axis(interpolate_column_galois, 0, R_galois)
W_polys = np.apply_along_axis(interpolate_column_galois, 0, O_galois)


# "evaluate" polynomials with the witness vector by inner product
def inner_product_polynomials_with_witness(polys, witness):
    mul_ = lambda x, y: x * y
    sum_ = lambda x, y: x + y
    return reduce(sum_, map(mul_, polys, witness))


# Compute QAP polynomials
sum_au = inner_product_polynomials_with_witness(U_polys, a)  # Ui(x)·ai
sum_av = inner_product_polynomials_with_witness(V_polys, a)  # Vi(x)·ai
sum_aw = inner_product_polynomials_with_witness(W_polys, a)  # Wi(x)·ai

# t(x) ensured that there are roots at every (x-i) from 1 to n
t = galois.Poly([1, curve_order - 1], field = GF)\
  * galois.Poly([1, curve_order - 2], field = GF)\
  * galois.Poly([1, curve_order - 3], field = GF)\
  * galois.Poly([1, curve_order - 4], field = GF)

# Evaluating at tau (only the trusted setup knows tau and should be destroyed)
t_tau = t(tau)

h = (sum_au * sum_av - sum_aw) // t
HT = h * t
 
# HT is equivalent to b(x) which is equivalent to the zero vector in polyonmial form
# Verify QAP equation holds does not have reaminded 

assert sum_au * sum_av == sum_aw + HT, "QAP division has remainder"

In [ ]:
# Trusted setup steps, generating secret values that the prover nor the verifier have access to, they should be random 

alpha = GF(433)  
beta = GF(534)   
tau = GF(825)    
gamma = GF(235)  
delta = GF(545)  

# SRS for G1
def SRS_tau_G1(tau):
    return [multiply(G1, int(tau ** i)) for i in range(t.degree)]

# SRS for G2
def SRS_tau_G2(tau):
    return [multiply(G2, int(tau ** i)) for i in range(t.degree)]

# SRS for h(tau)t(tau)
def generate_powers_of_tau_HT(tau):
    delta_inverse = GF(1) / delta
    before_delta_inverse = [multiply(G1, int(tau ** i * t_tau)) for i in range(t.degree - 1)]
    return [multiply(entry, int(delta_inverse)) for entry in before_delta_inverse]


# witness (psi_i)
# a*v1(tau), b*u1(tau)
alpha_times_V_polys = [alpha * V_polys[i] for i in range(len(V_polys))]
beta_times_U_polys = [beta * U_polys[i] for i in range(len(U_polys))]

# av1(tau)+bui(tau)+wi(tau)
C_polys = [beta_times_U_polys[i] + alpha_times_V_polys[i] + W_polys[i] for i in range(len(W_polys))]
C_polys_tau = [C_polys[i](tau) for i in range(len(C_polys))]
powers_of_tau_for_C = [multiply(G1, int(C_polys_tau[i])) for i in range(len(C_polys_tau))]

# public witness 
gamma_inverse = GF(1) / gamma
powers_of_tau_for_public_inputs = powers_of_tau_for_C[:l+1]
powers_of_tau_for_public_inputs = [multiply(entry, int(gamma_inverse)) for entry in powers_of_tau_for_public_inputs]

# private witness
gamma_inverse = GF(1) / delta
powers_of_tau_for_private_inputs = powers_of_tau_for_C[l+1:]
powers_of_tau_for_private_inputs = [multiply(entry, int(gamma_inverse)) for entry in powers_of_tau_for_private_inputs]

# group elements for verification
alpha_G1 = multiply(G1, int(alpha))
beta_G1 = multiply(G1, int(beta))
beta_G2 = multiply(G2, int(beta))
gamma_G2 = multiply(G2, int(gamma))
delta_G1 = multiply(G1, int(delta))
delta_G2 = multiply(G2, int(delta))


In [ ]:
# Prover steps
# generate "salt" to prevent an attacker from being able to guess the witness
r = GF(11)     
s = GF(22)    

def inner_product(ec_points, coeffs):
    return reduce(add, (multiply(point, int(coeff)) for point, coeff in zip(ec_points, coeffs)), Z1)


def encrypted_evaluation_G1(poly):
    powers_of_tau = SRS_tau_G1(tau)
    return inner_product(powers_of_tau, poly.coeffs[::-1])


def encrypted_evaluation_G2(poly):
    powers_of_tau = SRS_tau_G2(tau)
    return inner_product(powers_of_tau, poly.coeffs[::-1])


def encrypted_evaluation_HT(poly):
    powers_of_tau = generate_powers_of_tau_HT(tau)
    return inner_product(powers_of_tau, poly.coeffs[::-1])


first_A = encrypted_evaluation_G1(sum_au)
first_B1 = encrypted_evaluation_G1(sum_av)
first_B2 = encrypted_evaluation_G2(sum_av)

HT_tau = encrypted_evaluation_HT(h)
first_C = inner_product(powers_of_tau_for_private_inputs, private_inputs)

A1 = add(add(first_A, alpha_G1), multiply(delta_G1, int(r))) 
B2 = add(add(first_B2, beta_G2), multiply(delta_G2, int(s))) 
B1 = add(add(first_B1, beta_G1), multiply(delta_G1, int(s))) 
C1 = add(add(add(add(first_C, HT_tau), multiply(A1,int(s))), multiply(B1, int(r))), neg(multiply(delta_G1, int(r * s))))


# Final proof for verification 
proof = [A1, B2, C1]

In [92]:
# final verification

final_exponentiate(
    pairing(B2,neg(A1))*
    pairing(beta_G2, alpha_G1)*
    pairing(gamma_G2, inner_product(powers_of_tau_for_public_inputs, public_inputs))* # Step to verify that the public inputs in the witness have actually been used
    
    pairing(delta_G2, C1)
    ) == FQ12.one()

# Returns true, verification is complete

True